# Import Library

In [1]:
import sys
import os
try:
    from mechanize import Browser
except ImportError:
    !{sys.executable} - m pip install mechanize
    from mechanize import Browser

try:
    from bs4 import BeautifulSoup
except ImportError:
    !{sys.executable} - m pip install bs4
    from bs4 import BeautifulSoup

try:
    import time
except ImportError:
    !{sys.executable} - m pip install time
    import time

try:
    import pandas as pd
except ImportError:
    !{sys.executable} - m pip install pandas
    import pandas as pd

# Login to Timeform

In [2]:
tic = time.perf_counter()
started_time = time.time()
display("Login...")
browser = Browser()
browser.set_handle_robots(False)
browser.addheaders = [('User-agent', 'Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.9.0.1) Gecko/2008071615 Fedora/3.0.1-1.fc9 Firefox/3.0.1')]
browser.open('https://www.timeform.com/horse-racing/account/sign-in?returnUrl=%2Fhorse-racing%2F')
browser.select_form(nr=0)
browser["EmailAddress"] = 'routus2000@gmail.com'
browser["Password"] = 'number77A'
logged = browser.submit()
response = logged.read()

# you can delete this section:
elapsed_time = time.time() - started_time
display("Login complete. " + str(elapsed_time) + ' seconds')
soup = BeautifulSoup(response, "lxml")
html = soup

'Login...'

'Login complete. 63.71378540992737 seconds'

# Get Urls daily

In [3]:
day=10
month = 4
year = 2022

baseDateStr = f"{year}-{month:02d}-{day:02d}"
raceUrls = []
display("Search URLs - {date}.")
html = browser.open('https://www.timeform.com/horse-racing/results/' + baseDateStr).read()
soup = BeautifulSoup(html, "lxml")
for data in soup.find_all('a', class_='results-title'):
    print(data.get('href'))  # for getting link
    raceUrls.append(data.get('href'))

display("Searching complete.")

'Search URLs - {date}.'

/horse-racing/result/stratford-on-avon/2022-04-10/1350/48/1
/horse-racing/result/stratford-on-avon/2022-04-10/1423/48/2
/horse-racing/result/stratford-on-avon/2022-04-10/1455/48/3
/horse-racing/result/stratford-on-avon/2022-04-10/1530/48/4
/horse-racing/result/stratford-on-avon/2022-04-10/1605/48/5
/horse-racing/result/stratford-on-avon/2022-04-10/1640/48/6
/horse-racing/result/stratford-on-avon/2022-04-10/1710/48/7
/horse-racing/result/wincanton/2022-04-10/1300/56/1
/horse-racing/result/wincanton/2022-04-10/1330/56/2
/horse-racing/result/wincanton/2022-04-10/1405/56/3
/horse-racing/result/wincanton/2022-04-10/1440/56/4
/horse-racing/result/wincanton/2022-04-10/1515/56/5
/horse-racing/result/wincanton/2022-04-10/1550/56/6
/horse-racing/result/curragh/2022-04-10/1320/204/1
/horse-racing/result/curragh/2022-04-10/1355/204/2
/horse-racing/result/curragh/2022-04-10/1430/204/3
/horse-racing/result/curragh/2022-04-10/1505/204/4
/horse-racing/result/curragh/2022-04-10/1540/204/5
/horse-racing

'Searching complete.'

### Scrape results page for each race

In [4]:
raceList={
    "haydock-park":"Haydock",
    "lingfield-park":"Lingfield",
    "chelmsford-city":"Chelmsford City",
    "kempton-park":"Kempton",
    "ffos-las":"Ffos Las",
    "catterick-bridge":"Catterick",
    "sandown-park":"Sandown",
    "fontwell-park":"Fontwell",
    "market-rasen":"Market Rasen",
    "stratford-on-avon":"Stratford",
    "epsom-downs":"Epsom",
    "hamilton-park":"Hamilton",
#    "newmarket":"Newmarket (July)",
    "gowran-park":"Gowran Park",
    "newton-abbot":"Newton Abbot",
    }
    
display("Getting Race Result Data from Urls.")
allDataFrame = pd.DataFrame()
for raceUrl in raceUrls:
    url = 'https://www.timeform.com' + raceUrl
    display("Scraping Data from " + raceUrl + "...")
    x = url.split('/')
    location = x[5]
    if x[5] in raceList:
        location = raceList[x[5]]
    date = x[6][8:10] + '/' + x[6][5:7] + '/' + x[6][0:4]
    #print(date)
    dtime = x[7][:2] + ':' + x[7][2:]
    
    html = browser.open(url).read()
    soup = BeautifulSoup(html, "lxml")
    
    posArr = []
    positions = soup.find_all('span', attrs={'title': 'Finishing Position'})
    for position in positions:
        posArr.append(position.text)

    horseArr = []
    horses = soup.find_all('a', class_='rp-horse')
    for horse in horses:
        horseName = horse.text
        dotPos = horseName.find('.')
        horseName = horseName[dotPos+2:]
        bracketPos = horseName.find('(')
        if bracketPos > 0:
            horseName = horseName[:bracketPos-1]
        horseArr.append(horseName)

    winSpArr = []
    winSPs = soup.find_all('td', title='Betfair Win SP')
    for winSP in winSPs:
        if winSP.text != 'BSP':
            if len(winSP.text) > 1:
                winSpArr.append(float(winSP.text))
            else:
                winSpArr.append('0')

    plcSpArr=[]
    plcSPs = soup.find_all('td', title='Betfair Place SP')
    for plcSP in plcSPs:
        if plcSP.text != '(Place)':
            bracketLoc1 = plcSP.text.find('(')
            bracketLoc2 = plcSP.text.find(')')
            plcOdds = plcSP.text[bracketLoc1+1:bracketLoc2]
            if plcOdds:
                plcSpArr.append(plcOdds)
                plcSpArr.append(plcOdds)
            else:
                plcSpArr.append('0')
                plcSpArr.append('0')

    oneRaceData = pd.DataFrame(horseArr)
    oneRaceData = pd.DataFrame()
    oneRaceData['Horse'] = horseArr
    oneRaceData['Position'] = posArr
    oneRaceData['Betfair Win SP'] = winSpArr
    oneRaceData['Betfair Placed'] = ''
    oneRaceData['Betfair Place SP'] = plcSpArr
    
    oneRaceData = oneRaceData.drop_duplicates()
    oneRaceData['Date'] = date
    oneRaceData['Date'].values[:] = date
    oneRaceData['Meeting'] = location
    oneRaceData['Meeting'].values[:] = location
    oneRaceData['Time24Hour'] = dtime
    oneRaceData['Time24Hour'].values[:] = dtime

    oneRaceData = oneRaceData.set_index(['Date','Time24Hour', 'Meeting', 'Horse'])
    if raceUrl == raceUrls[0]:
        allDataFrame = oneRaceData
    else:
        allDataFrame = allDataFrame.append(oneRaceData, ignore_index=False)
    display("Complete Scraping Data from " + raceUrl + ".")
display("All Race Result Scraped.")

'Getting Race Result Data from Urls.'

'Scraping Data from /horse-racing/result/stratford-on-avon/2022-04-10/1350/48/1...'

'Complete Scraping Data from /horse-racing/result/stratford-on-avon/2022-04-10/1350/48/1.'

'Scraping Data from /horse-racing/result/stratford-on-avon/2022-04-10/1423/48/2...'

'Complete Scraping Data from /horse-racing/result/stratford-on-avon/2022-04-10/1423/48/2.'

...

'Scraping Data from /horse-racing/result/tramore/2022-04-10/1700/228/6...'

'Complete Scraping Data from /horse-racing/result/tramore/2022-04-10/1700/228/6.'

'Scraping Data from /horse-racing/result/tramore/2022-04-10/1730/228/7...'

'Complete Scraping Data from /horse-racing/result/tramore/2022-04-10/1730/228/7.'

'All Race Result Scraped.'

### Save Scrape results to csv

In [5]:
filename = 'ScrapData.csv'
allDataFrame.to_csv(filename)

# Merge Data

In [6]:
dateSumStr = baseDateStr.replace('-', '')
resultDataGlobal = pd.read_csv('ScrapData.csv').drop(columns=['Date'])
resultDataGlobal_ = resultDataGlobal.copy(deep=True)
dateStr = dateSumStr.replace('/', '')
pathName = r"../"

fileName = dateStr + ' ALL.csv'
csvPath = os.path.join(fileName)

ukhrData = pd.read_csv(csvPath)
ukhrData = ukhrData.drop(columns=['WRITE_BETFAIR_PLACED','WRITE_IN_RESULT_HERE','WRITE_BETFAIR_SP_HERE','WRITE_BETFAIR_PLACE_SP_HERE'])
ukhrData = ukhrData.set_index(['Date', 'Time24Hour', 'Meeting', 'Horse'])
filename = 'LatestResults_drop.csv'
ukhrData.to_csv(filename)

resultDataGlobal = resultDataGlobal_
resultDataGlobal = resultDataGlobal.reset_index()
resultDataGlobal['Horse'] = resultDataGlobal['Horse'].str.lower()
resultDataGlobal['Meeting'] = resultDataGlobal['Meeting'].str.lower()
resultDataGlobal['Horse'] = resultDataGlobal['Horse'].str.replace("`", "'")

ukhrData = ukhrData.reset_index()

horseList=[]
meetingList=[]
horseList=list(ukhrData['Horse'])
meetingList = list(ukhrData['Meeting'])
horseList_=horseList#.copy(deep=True)
meetingList_=meetingList

ukhrData['Horse'] = ukhrData['Horse'].str.lower()
ukhrData['Meeting'] = ukhrData['Meeting'].str.lower()
ukhrData['Horse'] = ukhrData['Horse'].str.replace("`", "'")

mergedDf = ukhrData.merge(resultDataGlobal, on=[ 'Time24Hour', 'Meeting', 'Horse'], how='left')
mergedDf.rename(columns={'date': 'Date'})
mergedDf.rename(columns={'meeting': 'Meeting'})
mergedDf.rename(columns={'horse': 'Horse'})
mergedDf.rename(columns={'time24hour': 'Time24Hour'})
nRows = mergedDf.shape[0]
for row in range(0, nRows-1):
    position = mergedDf.loc[row].at['Position']
    nRunners = mergedDf.loc[row].at['Runners']
    Handicap = mergedDf.loc[row].at['Handicap']
    mergedDf.at[row, 'Betfair Placed'] = ''
    if str(position).isnumeric():  # if position is a number
        if nRunners <= 7 and int(position) <= 2:
            mergedDf.at[row, 'Betfair Placed'] = '1'
        if nRunners >= 8 and int(position) <= 3:
            mergedDf.at[row, 'Betfair Placed'] = '1'
        if (Handicap == 'Handicap' or Handicap == 'Nursery') and nRunners >= 16 and int(position) <= 4:
            mergedDf.at[row, 'Betfair Placed'] = '1'

mergedDf = mergedDf.groupby(['Date','Time24Hour', 'Meeting', 'Horse'], as_index=False, sort=False).first()
mergedDf['Horse']=horseList_
mergedDf['Meeting']=meetingList_
mergedDf = mergedDf.reset_index()
mergedDf = mergedDf[list(mergedDf.columns)[1:]]
mergedDf = mergedDf.set_index(['Date'])

if 'index' in mergedDf.columns:
    mergedDf.drop('index', inplace=True, axis=1)

filename = dateStr + ' Data.csv'
mergedDf.to_csv(filename)

toc = time.perf_counter()
print(f"Total has {toc - tic:0.4f} seconds.")

NameError: name 'os' is not defined